In [63]:
# from docx import Document

# def split_docx_pages(input_docx_path, output_dir):
#     # Open the Word document
#     doc = Document(input_docx_path)

#     # Iterate over each section in the document
#     for i, section in enumerate(doc.sections):
#         # Create a new Document object for this page
#         new_doc = Document()

#         # Copy the content from the original document to the new document
#         for element in section.body:
#             new_doc._body.append(element)

#         # Save the new document to a file
#         new_doc_path = f"{output_dir}/page_{i + 1}.docx"
#         new_doc.save(new_doc_path)

#         print(f"Page {i + 1} saved as {new_doc_path}")



# # Example usage:
# input_docx_path = "New User Adoption Execution Steps.docx"
# output_dir = "output_pages"
# split_docx_pages(input_docx_path, output_dir)


In [6]:
!pip install aspose-words

  Obtaining dependency information for aspose-words from https://files.pythonhosted.org/packages/6e/7f/1beb1efa0766f6fc83b51ea7f3cd611d5b08c04a1cd6d457f013f7c8b20e/aspose_words-24.3.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/62.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/62.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/62.8 MB 1.8 MB/s eta 0:00:36
   ---------------------------------------- 0.3/62.8 MB 2.5 MB/s eta 0:00:26
   ---------------------------------------- 0.7/62.8 MB 3.8 MB/s eta 0:00:17
    --------------------------------------- 1.0/62.8 MB 4.4 MB/s eta 0:00:14
    --------------------------------------- 1.3/62.8 MB 4.7 MB/s eta 0:00:14
   - -------------------------------------- 1.6/62.8 MB 5.2 MB/s eta 0:00:12
   - -------------------------------------- 1.9/62.8 MB 5.2 MB/s eta 0:00:12
   - -------------------------------------- 2.3/62.8 MB 5.7 MB/s eta 0:00:11
   - -----------------------

In [68]:
import aspose.words as aw

# load Word document
doc = aw.Document("New User Adoption Execution Steps.docx")

# extract range of pages
extractedPages = doc.extract_pages(3, 6)

# save pages as a separate document
extractedPages.save("split_by_page_range.docx")

In [69]:
import aspose.words as aw

# load Word document
doc = aw.Document("New User Adoption Execution Steps.docx")

# get page count
pageCount = doc.page_count

# loop through pages
for page in range(0, pageCount):
  
    # save each page as a separate document
    extractedPage = doc.extract_pages(page, 1)
    extractedPage.save(f"split_by_page_{page + 1}.docx")

In [88]:
wiki_titles=["split_by_page_1","split_by_page_2","split_by_page_3","split_by_page_4","split_by_page_5","split_by_page_6","split_by_page_7","split_by_page_8","split_by_page_9"]

In [89]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
# from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

In [90]:
# # Load documents
# city_docs = []
# for wiki_title in wiki_titles:
#     docs = SimpleDirectoryReader(
#         input_files=[f"./{wiki_title}.docx"]  # Change the file extension to .doc or any other desired extension
#     ).load_data()
#     docs[0].doc_id = wiki_title
#     city_docs.extend(docs)
# import os
# import PyPDF2

# def load_pdf_files(pdf_files):
#     loaded_docs = []
#     for pdf_file in pdf_files:
#         with open(pdf_file, 'rb') as file:
#             pdf_reader = PyPDF2.PdfReader(file)
#             pdf_text = ""
#             for page in pdf_reader.pages:
#                 pdf_text += page.extract_text()
#             loaded_docs.append(pdf_text)
#     return loaded_docs

# # Example usage
# pdf_files = ['./file1.pdf', './file2.pdf']  # List of PDF file paths
# city_docs = load_pdf_files(pdf_files)
city_docs = []
for wiki_title in wiki_titles:
    docs = SimpleDirectoryReader(
        input_files=[f"./{wiki_title}.docx"]
    ).load_data()
    docs[0].doc_id = wiki_title
    city_docs.extend(docs)

In [79]:
splitter = SentenceSplitter(chunk_size=1000)

In [80]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [81]:
import nest_asyncio

nest_asyncio.apply()

In [91]:
# from langchain_together import Together
# splitter = SentenceSplitter(chunk_size=1000)
# llm= Together(
#         model="togethercomputer/llama-2-70b-chat",
#         # model="gpt-4",
#         temperature=0.1,
# #         max_tokens=207,
#         top_k=1,
#         together_api_key="1cecf43792b1187b044ab0293853353cc45caf8fdfe0a82e049d53cbf5954d26",
#     )
from llama_index.llms.together import TogetherLLM
llm = TogetherLLM(
    model="togethercomputer/llama-2-70b-chat", api_key="96557b956acf6073510ee7e4abadc1c7863626e75278a0eaf9a747875af30604"
)

In [92]:
# from llama_index.embeddings.together import TogetherEmbedding

# embed_model = TogetherEmbedding(
#     model_name="togethercomputer/m2-bert-80M-8k-retrieval", api_key="1cecf43792b1187b044ab0293853353cc45caf8fdfe0a82e049d53cbf5954d26")

from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
lc_embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
embed_model = LangchainEmbedding(lc_embed_model)
Settings.lc_embed_model = embed_model

In [93]:
from llama_index.core import Settings
Settings.context_window = 3000
Settings.llm =llm

In [94]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
# from sentence_transformers import SentenceTransformer, util
import os
from llama_index.core import Settings

Settings.llm =llm
# from llama_index.core import Settings
# Settings.context_window = 4096
# Settings.llm =llm
Settings.embed_model = embed_model

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

In [95]:
doc_summary_index = DocumentSummaryIndex.from_documents(
    city_docs,
    llm=llm,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/9 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/9 [00:00<?, ?it/s]

current doc id: split_by_page_1
current doc id: split_by_page_2
current doc id: split_by_page_3
current doc id: split_by_page_4
current doc id: split_by_page_5
current doc id: split_by_page_6
current doc id: split_by_page_7
current doc id: split_by_page_8
current doc id: split_by_page_9


Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

In [96]:
def get_document_summaries(document_names):
   
    document_summaries = []  # Initialize an empty list to store document summaries
    for doc_name in document_names:
        summary = doc_summary_index.get_document_summary(doc_name)  # Get summary for each document
        document_summaries.append(summary)  # Store the summary in the list
    return document_summaries

# Example usage:
document_names =["split_by_page_1","split_by_page_2","split_by_page_3","split_by_page_4","split_by_page_5","split_by_page_6","split_by_page_7","split_by_page_8","split_by_page_9"]
summaries = get_document_summaries(document_names)
print(summaries)

['  The provided text appears to be a set of instructions or steps for refreshing a dashboard related to user adoption. The text mentions taking backups of various dashboards and databases, and then downloading data in Excel format. It also mentions adding columns to the Excel sheet.\n\nSome questions that this text can answer include:\n\n* What are the steps involved in refreshing a user adoption dashboard?\n* How do you take backups of dashboards and databases?\n* How do you download data from a dashboard into Excel format?\n* How do you add columns to an Excel sheet?\n* What is the purpose of refreshing a user adoption dashboard?\n* What kind of data is used to refresh a user adoption dashboard?', '  The provided text appears to be a documentation or a report that discusses various business-related topics, including country names, column names, functions, and dashboards. It also mentions a software tool called Aspose.Words, which is used to create and manipulate Word documents.\n\nT

In [97]:
import re

def extract_questions_from_list(text_list):
    """
    Extract questions from a list of text strings.
    
    Args:
    text_list (list of str): List of text strings.
    
    Returns:
    list: List of extracted questions.
    """
    questions = []  # Initialize an empty list to store extracted questions
    pattern = r'\* (.+\?)'  # Regular expression pattern to match questions
    
    for text in text_list:
        extracted_questions = re.findall(pattern, text)  # Extract questions from each text string
        questions.extend(extracted_questions)  # Extend the list of questions with the extracted ones
    
    return questions


extracted_questions = extract_questions_from_list(summaries)
print(extracted_questions)

['What are the steps involved in refreshing a user adoption dashboard?', 'How do you take backups of dashboards and databases?', 'How do you download data from a dashboard into Excel format?', 'How do you add columns to an Excel sheet?', 'What is the purpose of refreshing a user adoption dashboard?', 'What kind of data is used to refresh a user adoption dashboard?', 'What are the different countries mentioned in the text?', 'What are the various functions and dashboards discussed in the text?', 'What is the purpose of the "Name hist view" column?', 'What is the difference between a cockpit and a non-cockpit dashboard?', 'What is the software tool Aspose.Words used for?', 'What is the purpose of the QARA Dashboard?', 'How do I filter data based on the local date and time in SQL?', 'What is the purpose of the Aspose.Words software, and what are its capabilities?', 'How do I download sales and RA dashboards for different regions?', 'What is the format of the data in the APAC_USAGE_HIST_SN

In [48]:
doc_summary_index.get_document_summary("split_by_page_1")

'\nThe provided text appears to be a set of instructions or guidelines for refreshing a dashboard related to user adoption. The text mentions various databases, tables, and queries that are used to take backups and download data in Excel format. Additionally, the text mentions three different regions: ANZ/SGMY/TH, KR/JP, and NPI.\n\nSome of the questions that this text can answer include:\n\n* What are the steps involved in refreshing the user adoption dashboard?\n* What databases and tables are used to retrieve data for the dashboard?\n* How is the data downloaded and manipulated to create the dashboard?\n* What regions are included in the dashboard and how are their data handled differently?\n* What is the purpose of the user adoption dashboard and what insights can it provide?'

In [ ]:
def get_document_summaries(document_names):
   
    document_summaries = []  # Initialize an empty list to store document summaries
    for doc_name in document_names:
        summary = doc_summary_index.get_document_summary(doc_name)  # Get summary for each document
        document_summaries.append(summary)  # Store the summary in the list
    return document_summaries

# Example usage:
wiki_titles=["split_by_page_1","split_by_page_2","split_by_page_3","split_by_page_4","split_by_page_5"]
summaries = get_document_summaries(document_names)
print(summaries)

In [49]:
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index")
doc_summary_index = load_index_from_storage(storage_context)

In [50]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

In [51]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage,MessageRole

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Summarize the documents.\n"
             "Always answer the query using the provided context information,"
        ),
    ), 
    ChatMessage(
        role=MessageRole.USER,
        content=(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information , "
    "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
query="""please summarize the documents"""

In [52]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query)

In [53]:
print(response__1)

 Sure, here's a summary of the document in bullet points:

• User adoption is used to check how many users have used the sales Genie Dashboard in the last one week.

• To refresh the dashboard, take backups of all three dashboards from Tableau and update the data in excel for all six regions.

• After downloading the data, open the excel sheet, delete the first column, and add two columns (Country & function) in each region.

• Filter the data according to created_at_local in SQL.

• Take a filtered data in excel and copy that data in a new excel sheet and save it.

• Open a remote desktop connection, go to SSMS, copy all excels, and paste them in the Bang Account remote desktop.

• Connect to the test environment, select the database (APACBI_FIGO_SALES), right-click, select Task, Import data, and choose a data source window.

• Select Data source as Micro soft excel, browse your excel sheet (single sheet at a time), click Next, and choose a Destination window.

• Select data base (SQL

In [54]:
query1="What are User Adoption Execution Steps? Summarize."

In [55]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query1)

In [56]:
print(response__1)

 Sure! Here are the User Adoption Execution Steps:

1. Take backups for all 3 dashboards from Tableau.
2. Connect to test database in SQL.
3. Take backups for all tables from the database.
4. Download data in excel for all 6 regions.
5. Open the excel sheet and delete the first column.
6. Add two columns (Country & function) in each region.
7. Remove 1 or 2 next to the QARA Dashboard in the NPI column.
8. Check for data created at local date and time.
9. Filter the data according to created_at_local in SQL.
10. Take a filtered data in excel and copy that data in a new excel sheet and save it.
11. Open the remote desktop connection and go to SSMS.
12. Copy all excels and paste in Bang Account remote desktop.
13. Go to SSMS and connect to the test environment.
14. Select Database (APACBI_FIGO_SALES) and right-click.
15. Select Task and Import data.
16. Choose a data source window and select Micro soft excel.
17. Browse your excel sheet (single sheet at a time) and click Next.
18. Choose 

In [57]:
query2="Which link I have to use for taking backup of dashboard?"

In [58]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query2)

In [59]:
print(response__1)



* Use the link <https://tableau-qa.medtronic.com/#/site/APACBI> to access the QA server.
* Click on the "Sales Genie" tab.
* Select the "Sales Genie user analytics" option for ANZ/SGMY/TH, "Sales Genie user analytics(JP_KR)" for KR/JP, and "NPI Genie User Anlaytics" for NPI.
* Click on the "Download" button to download the data in excel for all 6 regions.
* Use the link <https://tableau.medtronic.com/#/views/TableauUsageDetails_16226544435390/TraffictoViews?:iid=1> to download the data in excel for all 6 regions.

Please note that the provided links are for reference purposes only and may not be accessible outside of the Medtronic network.


In [60]:
query3=" Provide me details of “MDT Network Ids"

In [61]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query3)

In [62]:
print(response__1)

 Sure, here are the details of "MDT Network Ids" in bullet points:

* MDT Network Ids are used to identify and manage user adoption in the sales Genie Dashboard.
* The Ids are created and managed by the Medtronic IT team.
* There are four MDT Network Ids for different regions: ANZ/SGMY/TH, KR/JP, NPI, and Japan/Korea.
* Each Id has a unique name and description, such as "ANZ/SGMY/TH - Sales Genie User Analytics" and "KR/JP - Sales Genie User Analytics (JP_KR)".
* The Ids are used to filter and analyze user adoption data in the sales Genie Dashboard.
* The data is refreshed every Friday in QA, and the MDT Network Ids are used to identify and update the data for each region.
* The updated data is then loaded into the FIGO_SALES.ALL_SALES_GENIE_USERS table.
* The MDT Network Ids are also used to run stored procedures, such as dbo.APAC_Usage_Sales_Genie, to update the Comex IT flag for users.
* The flag is updated based on the user's country and region.
* The updated flag is then reflected

In [98]:
query_1='What are the steps involved in refreshing a user adoption dashboard?'
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__1)

  Sure! Here are the steps involved in refreshing a user adoption dashboard:

• Take backups for all three dashboards from Tableau.
• Connect to the test database in SQL.
• Take backups for all tables from the database.
• For ANZ/SGMY/TH, use two queries to take a backup.
• For KR/JP, use two queries to take a backup.
• For NPI, use one query to take a backup.
• Download the data in Excel for all six regions using a link.
• Open the Excel sheet and delete the first column.
• Add two columns (Country & Function) in each region before Column ‘K’.

I hope that helps! Let me know if you have any further questions.


In [99]:
query_2='How do you take backups of dashboards and databases?'
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__2)

  Sure! Here are the steps involved in refreshing a user adoption dashboard:

• Take backups for all three dashboards from Tableau.
• Connect to the test database in SQL.
• Take backups for all tables from the database.
• For ANZ/SGMY/TH, use two queries to take a backup.
• For KR/JP, use two queries to take a backup.
• For NPI, use one query to take a backup.
• Download the data in excel for all six regions using the provided link.
• Open the excel sheet and delete the first column.
• Add two columns (Country & Function) in each region.
• The first step in refreshing the user adoption dashboard is taking backups for all three dashboards from Tableau.
• The second step is connecting to the test database in SQL.
• The third step is taking backups for all tables from the database.
• The fourth step is using two queries to take a backup for ANZ/SGMY/TH.
• The fifth step is using two queries to take a backup for KR/JP.
• The sixth step is using one query to take a backup for NPI.
• The sev

In [100]:
query_2='How do you download data from a dashboard into Excel format?'
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__2)

  Sure! Here are the steps involved in refreshing a user adoption dashboard:

• Take backups for all three dashboards from Tableau.
• Connect to the test database in SQL.
• Take backups for all tables from the database.
• For ANZ/SGMY/TH, use two queries to take a backup.
• For KR/JP, use two queries to take a backup.
• For NPI, use one query to take a backup.
• Download the data in Excel for all six regions using a link.
• Open the Excel sheet and delete the first column.
• Add two columns (Country & Function) in each region before Column ‘K’.

I hope this helps! Let me know if you have any further questions.


In [101]:

query_2='How do you add columns to an Excel sheet?'
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__2)

  Sure! Here are the steps involved in refreshing a user adoption dashboard:

• Take backups for all three dashboards from Tableau.
• Connect to the test database in SQL.
• Take backups for all tables from the database.
• For ANZ/SGMY/TH, use two queries to take a backup.
• For KR/JP, use two queries to take a backup.
• For NPI, use one query to take a backup.
• Download the data in excel for all six regions using a link.
• Open the excel sheet and delete the first column.
• Add two columns (Country & Function) in each region.

I hope this helps! Let me know if you have any further questions.


In [102]:
query_2='What is the purpose of refreshing a user adoption dashboard?'

response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__2)

  Sure! Here are the steps involved in refreshing a user adoption dashboard:

• Take backups for all three dashboards from Tableau.
• Connect to the test database in SQL.
• Take backups for all tables from the database.
• For ANZ/SGMY/TH, use two queries to take a backup.
• For KR/JP, use two queries to take a backup.
• For NPI, use one query to take a backup.
• Download the data in Excel for all six regions using a link.
• Open the Excel sheet and delete the first column.
• Add two columns (Country & Function) in each region before Column ‘K’.

I hope this helps! Let me know if you have any further questions.


In [103]:
query_2='What kind of data is used to refresh a user adoption dashboard?'

response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__2)

  Sure! Here are the steps involved in refreshing a user adoption dashboard:

• Take backups for all three dashboards from Tableau.
• Connect to the test database in SQL.
• Take backups for all tables from the database.
• For ANZ/SGMY/TH, use two queries to take a backup.
• For KR/JP, use two queries to take a backup.
• For NPI, use one query to take a backup.
• Download the data in Excel for all six regions using a link.
• Open the Excel sheet and delete the first column.
• Add two columns (Country & Function) in each region before Column ‘K’.

I hope this helps! Let me know if you have any further questions.


In [104]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage,MessageRole

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Summarize the documents.\n"
             "Always answer the query using the provided context information, "
            

        ),
    ), 
    ChatMessage(
        role=MessageRole.USER,
        content=(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information , "
    "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)


In [105]:
query="""'What are the different countries mentioned in the text?', 
•	'What are the various functions and dashboards discussed in the text?', 
•	'What is the purpose of the "Name hist view" column?', 
•	'What is the difference between a cockpit and a non-cockpit dashboard?',
•	'What is the software tool Aspose.Words used for?',"""

In [106]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query)

In [107]:
print(response__1)

  Sure, here are the answers to the questions:

•	What are the different countries mentioned in the text?

	ANZ
	APAC

•	What are the various functions and dashboards discussed in the text?

	Sales Dashboard
	RA Dashboard

•	What is the purpose of the "Name hist view" column?

	To check for data created at local date and time

•	What is the difference between a cockpit and a non-cockpit dashboard?

	Not mentioned in the text

•	What is the software tool Aspose.Words used for?

	Creating and editing Word documents


In [108]:
query="""'What is the purpose of the QARA Dashboard?',
'How do I filter data based on the local date and time in SQL?', 
'What is the purpose of the Aspose.Words software, and what are its capabilities?',
'How do I download sales and RA dashboards for different regions?',
'What is the format of the data in the APAC_USAGE_HIST_SNAPSHOT_ALL_QTR table, and what information does it contain?',"""

In [109]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query)
print(response__1)

  Sure, here are the answers to the questions:

What is the purpose of the QARA Dashboard?

* The QARA Dashboard is used to display and analyze data related to sales and RA (Returns, Allowances, and Adjustments) for different regions.

How do I filter data based on the local date and time in SQL?

* To filter data based on the local date and time in SQL, you can use the convert() function to convert the created_at_local column to a datetime data type, and then use the order by clause to sort the data in descending order based on the created_at_local column.

What is the purpose of the Aspose.Words software, and what are its capabilities?

* Aspose.Words is a software that allows you to create, modify, and convert Word documents programmatically. It can be used to generate documents, extract data from documents, and perform various other operations on Word documents.

How do I download sales and RA dashboards for different regions?

* To download sales and RA dashboards for different re

In [110]:
query_1="""'How do I use the convert() function in SQL to convert a date and time field to a datetime data type?'What is the difference between the All Business and RA dashboards, and how do they relate to the data in the APAC_USAGE_HIST_SNAPSHOT_ALL_QTR table?',
•	'What are the steps involved in preparing excel sheets for different regions?', 
•	'How do you copy and paste data into a remote desktop using SSMS?',
•	'What is the procedure for importing data into a database using SSMS?', 
•	'What software tools are required to carry out the task described in the text?',
•	'What is the purpose of the task described in the text?',
•	'How do I select a data source for my Aspose.Words project?',"""
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__1)

  •	To use the convert() function in SQL to convert a date and time field to a datetime data type, you can use the following syntax: convert(datetime, created_at_local)

•	The All Business dashboard and RA dashboard are both used to view data related to business performance, but the RA dashboard is more focused on providing insights into specific regions, whereas the All Business dashboard provides a broader overview of business performance across all regions. The data in the APAC_USAGE_HIST_SNAPSHOT_ALL_QTR table is related to the RA dashboard, as it contains data related to usage patterns in the APAC region.

•	Preparing excel sheets for different regions involves selecting the appropriate data for that region, filtering and sorting the data as needed, and then creating the desired visualizations and reports.

•	To copy and paste data into a remote desktop using SSMS, you can use the "Copy with headers" option in the SSMS query editor to copy the data to the clipboard, and then use t

In [111]:
query_1="""'What are the steps to connect to a SQL Server Native Client 11.0 database using Aspose.Words?',
•	'How do I view source tables and views in Aspose.Words?',
•	'What are the different options available for selecting a destination window in Aspose.Words?', 
•	'Is there a limit to the number of sheets that can be selected from a single Excel file in Aspose.Word?
•	'What are the different regions and their corresponding data sources that can be used with Aspose.Words?',
•	'How do you retrieve data for different regions using Aspose.Words?',"""
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__1)

  •	To connect to a SQL Server Native Client 11.0 database using Aspose.Words, follow these steps:

	1.	Select Data source as Microsoft Excel.
	2.	Browse your excel sheet (single sheet at a time).
	3.	Click Next.
	4.	Select data base (SQL Server Native client 11.0).
	5.	Check for server name it should be Test Environment.
	6.	Click Next.

•	To view source tables and views in Aspose.Words, follow these steps:

	1.	Click Next.
	2.	Source Tables and Views Window will appear.

•	The different options available for selecting a destination window in Aspose.Words are:

	1.	Microsoft Excel.

•	There is no limit to the number of sheets that can be selected from a single Excel file in Aspose.Words.

•	The different regions and their corresponding data sources that can be used with Aspose.Words are:

	1.	Data source as Microsoft Excel.

•	To retrieve data for different regions using Aspose.Words, follow these steps:

	1.	Select the desired region.
	2.	Click Next.
	3.	Select data base (SQL Server 

In [112]:
query_1="""'What is the purpose of the "Column Mappings Window" in Aspose.Words?', 
•	'How do you ensure that all columns are mapped correctly in Aspose.Words?', 
•	'What is the full version of Aspose.Words and how can you obtain it?',
•	'What is the purpose of the APAC_Usage_Sales_Genie stored procedure?', 
•	'Which dashboards need to be refreshed, and how are they related to user adoption?',
•	'What is the role of the Comex IT flag in this workflow?',"""
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__1)

  Sure! Here are the answers to the questions:

•	The purpose of the "Column Mappings Window" in Aspose.Words is to allow users to check whether all columns are mapped correctly.

•	To ensure that all columns are mapped correctly in Aspose.Words, users can click Ok after verifying that the columns are correctly mapped in the Column Mappings Window.

•	The full version of Aspose.Words can be obtained by visiting the Aspose website and discovering the full versions of their APIs.

•	The purpose of the APAC_Usage_Sales_Genie stored procedure is not specified in the given context information.

•	The dashboards that need to be refreshed are not specified in the given context information. However, it is mentioned that the ANZ/TH/SGMY usage dashboard needs to be refreshed.

•	The Comex IT flag is not specified in the given context information. Therefore, its role in this workflow is unknown.

It's worth noting that the provided context information is limited to a single page of a document, an

In [113]:
query_1="""Which countries or regions are mentioned in the text, and how are they related to the workflow?
•'What is the purpose of the FIGO_SALES.ALL_SALES_GENIE_USERS table, and how does it relate to the workflow?', 
•'What is Aspose.Words and what is it used for?', 
•'What are some of the features and capabilities of Aspose.Words?', 
•'How can I learn more about Aspose.Words and its full version?', 
•'Is there a free evaluation copy of Aspose.Words available?',
•'What kind of documents can Aspose.Words handle and manipulate?'"""
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_1)
print(response__1)

  Sure! Here are the answers to the questions you asked:

•	Japan, Korea, ANZ, TH, SG, and MY are the countries or regions mentioned in the text. They are related to the workflow as they are the destinations that can be selected for usage.

•	The purpose of the FIGO_SALES.ALL_SALES_GENIE_USERS table is not specified in the given text. However, based on the name of the table, it appears to be related to sales data or user information for a genie usage. It is mentioned in the context of column mappings, which suggests that it may be used in the workflow to map columns from one data source to another.

•	Aspose.Words is a software tool that allows users to create, modify, and manipulate Microsoft Word documents programmatically. It is developed by Aspose Pty Ltd and has been available since 2003.

•	Aspose.Words has a wide range of features and capabilities, including the ability to read and write Word documents in various formats, create and edit documents from scratch, add text, images,